## Import smp

In [1]:
pip install segmentation-models-pytorch

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 50.8 MB/s eta 0:00:0000:01
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16427 sha256=fa533c13f4c41a4b1114359c18a46622f8e6134614ba4fe90b17bfcded8d85a2
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60945 sha256=9be1065061a27de72c36ba0f0343d0bb68f0d99adc00d77f2768704c0695622d
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c60142835bfc889f9a482e4a67e0b817032d9c6883b64
Successfully built efficientnet-py

## Dataset

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import os
from pathlib import Path
import numpy as np
from tqdm import tqdm
import segmentation_models_pytorch as smp
import matplotlib.pyplot as plt

class HuronDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = Path(image_dir)
        self.mask_dir = Path(mask_dir)
        self.transform = transform
        self.mask_transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor()
        ])
        self.images = sorted(os.listdir(image_dir))
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = self.image_dir / self.images[idx]
        mask_path = self.mask_dir / self.images[idx]
        
        image = Image.open(img_path).convert('RGB')
        mask = Image.open(mask_path).convert('L')
        
        if self.transform:
            image = self.transform(image)
        mask = self.mask_transform(mask)
        
        # Ensure mask is binary
        mask = (mask > 0.5).float()
        
        return image, mask


## Eval and Metrics

In [ ]:
def calculate_iou(outputs, targets, threshold=0.5):
    outputs = (outputs > threshold).float()
    targets = targets.float()
    
    intersection = (outputs * targets).sum(dim=(1, 2))
    union = outputs.sum(dim=(1, 2)) + targets.sum(dim=(1, 2)) - intersection
    
    iou = (intersection + 1e-6) / (union + 1e-6)
    return iou.mean()

def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    total_iou = 0
    num_batches = len(dataloader)
    
    with torch.no_grad():
        for images, masks in dataloader:
            images = images.to(device)
            masks = masks.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, masks)
            iou = calculate_iou(outputs, masks)
            
            total_loss += loss.item()
            total_iou += iou.item()
    
    return total_loss / num_batches, total_iou / num_batches

## Training

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=5):
    best_val_iou = 0
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_iou = 0.0
        
        with tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}') as pbar:
            for images, masks in pbar:
                images = images.to(device)
                masks = masks.to(device)
                
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, masks)
                iou = calculate_iou(outputs, masks)
                
                loss.backward()
                optimizer.step()
                
                running_loss += loss.item()
                running_iou += iou.item()
                
                pbar.set_postfix({
                    'loss': running_loss / (pbar.n + 1),
                    'IoU': running_iou / (pbar.n + 1)
                })
        
        # Validate
        model.eval()
        val_iou = 0
        with torch.no_grad():
            for images, masks in val_loader:
                images = images.to(device)
                masks = masks.to(device)
                outputs = model(images)
                val_iou += calculate_iou(outputs, masks).item()
        
        val_iou /= len(val_loader)
        print(f'Validation IoU: {val_iou:.4f}')
        
        if val_iou > best_val_iou:
            best_val_iou = val_iou
            torch.save(model.state_dict(), 'best_model.pth')


## Unet

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import os
from pathlib import Path
import numpy as np
from tqdm import tqdm
import segmentation_models_pytorch as smp

def main():
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Dataset paths
    image_dir = '/kaggle/input/huron-dataset/Sliced_Images'
    mask_dir = '/kaggle/input/huron-dataset/Sliced_masks'
    
    # Transform for input images
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                           std=[0.229, 0.224, 0.225])
    ])
    
    # Create full dataset
    full_dataset = HuronDataset(image_dir, mask_dir, transform=transform)
    
    # Calculate splits
    total_size = len(full_dataset)
    train_size = int(0.7 * total_size)
    val_size = int(0.15 * total_size)
    test_size = total_size - train_size - val_size
    
    # Split dataset
    train_dataset, val_dataset, test_dataset = random_split(
        full_dataset, 
        [train_size, val_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )
    
    print(f"Dataset splits: Train={len(train_dataset)}, Val={len(val_dataset)}, Test={len(test_dataset)}")
    
    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=4)
    
    # Model configuration
    encoder_name = "resnet34"
    
    # Initialize U-Net model
    model = smp.Unet(
        encoder_name=encoder_name,        # try different encoders
        encoder_weights="imagenet",     # use `imagenet` pre-trained weights
        in_channels=3,                  # model input channels
        classes=1                      # model output channels
    ).to(device)
    
    print(f"Model: U-Net with {encoder_name} encoder")
    
    # Initialize loss and optimizer
    criterion = smp.losses.DiceLoss('binary')
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Train the model
    train_model(model, train_loader, val_loader, criterion, optimizer, device)
    
    # Evaluate on test set
    print("Evaluating on test set...")
    model.load_state_dict(torch.load('best_model.pth'))
    test_loss, test_iou = evaluate(model, test_loader, criterion, device)
    print(f'Test Loss: {test_loss:.4f}, Test IoU: {test_iou:.4f}')

if __name__ == '__main__':
    main()

Using device: cuda
Dataset splits: Train=12142, Val=2602, Test=2603
Model: U-Net with resnet34 encoder


Epoch 1/5: 100%|██████████| 1518/1518 [03:08<00:00,  8.06it/s, loss=0.123, IoU=0.871]


Validation IoU: 0.8721


Epoch 2/5: 100%|██████████| 1518/1518 [03:15<00:00,  7.77it/s, loss=0.09, IoU=0.894]  


Validation IoU: 0.8953


...

Validation IoU: 0.8991


Epoch 5/5: 100%|██████████| 1518/1518 [03:15<00:00,  7.75it/s, loss=0.0848, IoU=0.899]


Validation IoU: 0.9001
Evaluating on test set...


/tmp/ipykernel_30/3598702032.py:75: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


Test Loss: 0.0840, Test IoU: 0.9016


## Unet with SCSE

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import os
from pathlib import Path
import numpy as np
from tqdm import tqdm
import segmentation_models_pytorch as smp

def main():
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Dataset paths
    image_dir = '/kaggle/input/huron-dataset/Sliced_Images'
    mask_dir = '/kaggle/input/huron-dataset/Sliced_masks'
    
    # Transform for input images
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                           std=[0.229, 0.224, 0.225])
    ])
    
    # Create full dataset
    full_dataset = HuronDataset(image_dir, mask_dir, transform=transform)
    
    # Calculate splits
    total_size = len(full_dataset)
    train_size = int(0.7 * total_size)
    val_size = int(0.15 * total_size)
    test_size = total_size - train_size - val_size
    
    # Split dataset
    train_dataset, val_dataset, test_dataset = random_split(
        full_dataset, 
        [train_size, val_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )
    
    print(f"Dataset splits: Train={len(train_dataset)}, Val={len(val_dataset)}, Test={len(test_dataset)}")
    
    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=4)
    
    # Model configuration
    encoder_name = "resnet34"
    
    # Initialize Attention U-Net model
    model = smp.Unet(
        encoder_name=encoder_name,        # try different encoders
        encoder_weights="imagenet",     # use `imagenet` pre-trained weights
        in_channels=3,                  # model input channels
        classes=1,                      # model output channels
        decoder_attention_type="scse"   # spatial and channel squeeze & excitation
    ).to(device)
    
    print(f"Model: Attention U-Net with {encoder_name} encoder and SCSE attention")
    
    # Initialize loss and optimizer
    criterion = smp.losses.DiceLoss('binary')
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Train the model
    train_model(model, train_loader, val_loader, criterion, optimizer, device)
    
    # Evaluate on test set
    print("Evaluating on test set...")
    model.load_state_dict(torch.load('best_model.pth'))
    test_loss, test_iou = evaluate(model, test_loader, criterion, device)
    print(f'Test Loss: {test_loss:.4f}, Test IoU: {test_iou:.4f}')

if __name__ == '__main__':
    main()

Using device: cuda
Dataset splits: Train=12142, Val=2602, Test=2603
Model: Attention U-Net with resnet34 encoder and SCSE attention


Epoch 1/5: 100%|██████████| 1518/1518 [03:52<00:00,  6.52it/s, loss=0.128, IoU=0.866]


Validation IoU: 0.8909


Epoch 2/5: 100%|██████████| 1518/1518 [03:52<00:00,  6.53it/s, loss=0.0908, IoU=0.892]


Validation IoU: 0.9014


...

Validation IoU: 0.9026


Epoch 5/5: 100%|██████████| 1518/1518 [03:52<00:00,  6.54it/s, loss=0.0872, IoU=0.897]


Validation IoU: 0.9020
Evaluating on test set...


/tmp/ipykernel_30/1385039598.py:76: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


Test Loss: 0.0836, Test IoU: 0.9023


## Unet++

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import os
from pathlib import Path
import numpy as np
from tqdm import tqdm
import segmentation_models_pytorch as smp

def main():
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Dataset paths
    image_dir = '/kaggle/input/huron-dataset/Sliced_Images'
    mask_dir = '/kaggle/input/huron-dataset/Sliced_masks'
    
    # Transform for input images
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                           std=[0.229, 0.224, 0.225])
    ])
    
    # Create full dataset
    full_dataset = HuronDataset(image_dir, mask_dir, transform=transform)
    
    # Calculate splits
    total_size = len(full_dataset)
    train_size = int(0.7 * total_size)
    val_size = int(0.15 * total_size)
    test_size = total_size - train_size - val_size
    
    # Split dataset
    train_dataset, val_dataset, test_dataset = random_split(
        full_dataset, 
        [train_size, val_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )
    
    print(f"Dataset splits: Train={len(train_dataset)}, Val={len(val_dataset)}, Test={len(test_dataset)}")
    
    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=4)
    
    # Model configuration
    encoder_name = "resnet34"
    
    # Initialize Attention U-Net model
    model = smp.UnetPlusPlus(
        encoder_name=encoder_name,        # try different encoders
        encoder_weights="imagenet",     # use `imagenet` pre-trained weights
        in_channels=3,                  # model input channels
        classes=1                     # model output channel
    ).to(device)
    
    print(f"Model: U-Net++ with {encoder_name}")
    
    # Initialize loss and optimizer
    criterion = smp.losses.DiceLoss('binary')
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Train the model
    train_model(model, train_loader, val_loader, criterion, optimizer, device)
    
    # Evaluate on test set
    print("Evaluating on test set...")
    model.load_state_dict(torch.load('best_model.pth'))
    test_loss, test_iou = evaluate(model, test_loader, criterion, device)
    print(f'Test Loss: {test_loss:.4f}, Test IoU: {test_iou:.4f}')

if __name__ == '__main__':
    main()

Using device: cuda
Dataset splits: Train=12142, Val=2602, Test=2603
Model: U-Net++ with resnet34


Epoch 1/5: 100%|██████████| 1518/1518 [06:44<00:00,  3.76it/s, loss=0.116, IoU=0.874]


Validation IoU: 0.8813


Epoch 2/5: 100%|██████████| 1518/1518 [06:42<00:00,  3.77it/s, loss=0.0897, IoU=0.896]


Validation IoU: 0.9010


...

Validation IoU: 0.8945


Epoch 5/5: 100%|██████████| 1518/1518 [06:43<00:00,  3.76it/s, loss=0.0809, IoU=0.903]


Validation IoU: 0.8954
Evaluating on test set...


/tmp/ipykernel_30/2377843142.py:75: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


Test Loss: 0.0835, Test IoU: 0.9012


## Unet++ with SCSE

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import os
from pathlib import Path
import numpy as np
from tqdm import tqdm
import segmentation_models_pytorch as smp

def main():
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Dataset paths
    image_dir = '/kaggle/input/huron-dataset/Sliced_Images'
    mask_dir = '/kaggle/input/huron-dataset/Sliced_masks'
    
    # Transform for input images
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                           std=[0.229, 0.224, 0.225])
    ])
    
    # Create full dataset
    full_dataset = HuronDataset(image_dir, mask_dir, transform=transform)
    
    # Calculate splits
    total_size = len(full_dataset)
    train_size = int(0.7 * total_size)
    val_size = int(0.15 * total_size)
    test_size = total_size - train_size - val_size
    
    # Split dataset
    train_dataset, val_dataset, test_dataset = random_split(
        full_dataset, 
        [train_size, val_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )
    
    print(f"Dataset splits: Train={len(train_dataset)}, Val={len(val_dataset)}, Test={len(test_dataset)}")
    
    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=4)
    
    # Model configuration
    encoder_name = "resnet34"
    
    # Initialize Attention U-Net model
    model = smp.UnetPlusPlus(
        encoder_name=encoder_name,        # try different encoders
        encoder_weights="imagenet",     # use `imagenet` pre-trained weights
        in_channels=3,                  # model input channels
        classes=1,                      # model output channels
        decoder_attention_type="scse"   # spatial and channel squeeze & excitation
    ).to(device)
    
    print(f"Model: Attention U-Net++ with {encoder_name} encoder and SCSE attention")
    
    # Initialize loss and optimizer
    criterion = smp.losses.DiceLoss('binary')
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Train the model
    train_model(model, train_loader, val_loader, criterion, optimizer, device)
    
    # Evaluate on test set
    print("Evaluating on test set...")
    model.load_state_dict(torch.load('best_model.pth'))
    test_loss, test_iou = evaluate(model, test_loader, criterion, device)
    print(f'Test Loss: {test_loss:.4f}, Test IoU: {test_iou:.4f}')

if __name__ == '__main__':
    main()

Using device: cuda
Dataset splits: Train=12142, Val=2602, Test=2603


Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 187MB/s] 


Model: Attention U-Net++ with resnet34 encoder and SCSE attention


Epoch 1/5: 100%|██████████| 1518/1518 [08:55<00:00,  2.83it/s, loss=0.115, IoU=0.872]


Validation IoU: 0.8619


...

Validation IoU: 0.9026


Epoch 5/5: 100%|██████████| 1518/1518 [08:58<00:00,  2.82it/s, loss=0.0871, IoU=0.897]


Validation IoU: 0.9072
Evaluating on test set...


/tmp/ipykernel_30/782100786.py:76: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


Test Loss: 0.0782, Test IoU: 0.9068


## DeepLabV3+

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import os
from pathlib import Path
import numpy as np
from tqdm import tqdm
import segmentation_models_pytorch as smp

def main():
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Dataset paths
    image_dir = '/kaggle/input/huron-dataset/Sliced_Images'
    mask_dir = '/kaggle/input/huron-dataset/Sliced_masks'
    
    # Transform for input images
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                           std=[0.229, 0.224, 0.225])
    ])
    
    # Create full dataset
    full_dataset = HuronDataset(image_dir, mask_dir, transform=transform)
    
    # Calculate splits
    total_size = len(full_dataset)
    train_size = int(0.7 * total_size)
    val_size = int(0.15 * total_size)
    test_size = total_size - train_size - val_size
    
    # Split dataset
    train_dataset, val_dataset, test_dataset = random_split(
        full_dataset, 
        [train_size, val_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )
    
    print(f"Dataset splits: Train={len(train_dataset)}, Val={len(val_dataset)}, Test={len(test_dataset)}")
    
    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=4)
    
    # Model configuration
    encoder_name = "resnet34"
    
    # Initialize Attention U-Net model
    # Initialize DeepLabV3+ model
    model = smp.DeepLabV3Plus(
        encoder_name="resnet34",      # Using ResNet34 for faster training
        encoder_weights="imagenet",    # Using pretrained weights
        in_channels=3,                # RGB images
        classes=1,                    # Binary segmentation
    ).to(device)
    
    print(f"Model: DeepLabV3Plus with {encoder_name} encoder")
    
    # Initialize loss and optimizer
    criterion = smp.losses.DiceLoss('binary')
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Train the model
    train_model(model, train_loader, val_loader, criterion, optimizer, device)
    
    # Evaluate on test set
    print("Evaluating on test set...")
    model.load_state_dict(torch.load('best_model.pth'))
    test_loss, test_iou = evaluate(model, test_loader, criterion, device)
    print(f'Test Loss: {test_loss:.4f}, Test IoU: {test_iou:.4f}')

if __name__ == '__main__':
    main()

Using device: cuda
Dataset splits: Train=12142, Val=2602, Test=2603
Model: DeepLabV3Plus with resnet34 encoder


Epoch 1/5: 100%|██████████| 1518/1518 [03:12<00:00,  7.89it/s, loss=0.116, IoU=0.874]


Validation IoU: 0.8915


Epoch 2/5: 100%|██████████| 1518/1518 [03:11<00:00,  7.91it/s, loss=0.0941, IoU=0.89] 


Validation IoU: 0.8777


...

Validation IoU: 0.8986


Epoch 5/5: 100%|██████████| 1518/1518 [03:11<00:00,  7.93it/s, loss=0.0833, IoU=0.899]


Validation IoU: 0.8962
Evaluating on test set...


/tmp/ipykernel_30/1907528388.py:76: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


Test Loss: 0.0861, Test IoU: 0.8986


## MANet

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import os
from pathlib import Path
import numpy as np
from tqdm import tqdm
import segmentation_models_pytorch as smp

def main():
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Dataset paths
    image_dir = '/kaggle/input/huron-dataset/Sliced_Images'
    mask_dir = '/kaggle/input/huron-dataset/Sliced_masks'
    
    # Transform for input images
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                           std=[0.229, 0.224, 0.225])
    ])
    
    # Create full dataset
    full_dataset = HuronDataset(image_dir, mask_dir, transform=transform)
    
    # Calculate splits
    total_size = len(full_dataset)
    train_size = int(0.7 * total_size)
    val_size = int(0.15 * total_size)
    test_size = total_size - train_size - val_size
    
    # Split dataset
    train_dataset, val_dataset, test_dataset = random_split(
        full_dataset, 
        [train_size, val_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )
    
    print(f"Dataset splits: Train={len(train_dataset)}, Val={len(val_dataset)}, Test={len(test_dataset)}")
    
    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=4)
    
    # Model configuration
    encoder_name = "resnet34"
    
    # Initialize Attention U-Net model
    # Initialize DeepLabV3+ model
    model = smp.MAnet(
        encoder_name="resnet34",        # Using ResNet34 as encoder
        encoder_weights="imagenet",      # Using pretrained weights
        in_channels=3,                   # RGB images
        classes=1,                       # Binary segmentation
        decoder_channels=(256, 128, 64, 32, 16),  # Decoder channel sizes
    ).to(device)
    
    print(f"Model: MA-Net with {encoder_name} encoder")
    
    # Initialize loss and optimizer
    criterion = smp.losses.DiceLoss('binary')
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Train the model
    train_model(model, train_loader, val_loader, criterion, optimizer, device)
    
    # Evaluate on test set
    print("Evaluating on test set...")
    model.load_state_dict(torch.load('best_model.pth'))
    test_loss, test_iou = evaluate(model, test_loader, criterion, device)
    print(f'Test Loss: {test_loss:.4f}, Test IoU: {test_iou:.4f}')

if __name__ == '__main__':
    main()

Using device: cuda
Dataset splits: Train=12142, Val=2602, Test=2603
Model: MA-Net with resnet34 encoder


Epoch 1/5: 100%|██████████| 1518/1518 [03:34<00:00,  7.06it/s, loss=0.121, IoU=0.871]


Validation IoU: 0.8787


Epoch 2/5: 100%|██████████| 1518/1518 [03:35<00:00,  7.06it/s, loss=0.0919, IoU=0.891]


Validation IoU: 0.8988


...

Validation IoU: 0.9004


Epoch 5/5: 100%|██████████| 1518/1518 [03:34<00:00,  7.08it/s, loss=0.0838, IoU=0.9]  


Validation IoU: 0.5829
Evaluating on test set...


/tmp/ipykernel_30/202613861.py:77: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


Test Loss: 0.0858, Test IoU: 0.9001


## PAN

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import os
from pathlib import Path
import numpy as np
from tqdm import tqdm
import segmentation_models_pytorch as smp

def main():
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Dataset paths
    image_dir = '/kaggle/input/huron-dataset/Sliced_Images'
    mask_dir = '/kaggle/input/huron-dataset/Sliced_masks'
    
    # Transform for input images
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                           std=[0.229, 0.224, 0.225])
    ])
    
    # Create full dataset
    full_dataset = HuronDataset(image_dir, mask_dir, transform=transform)
    
    # Calculate splits
    total_size = len(full_dataset)
    train_size = int(0.7 * total_size)
    val_size = int(0.15 * total_size)
    test_size = total_size - train_size - val_size
    
    # Split dataset
    train_dataset, val_dataset, test_dataset = random_split(
        full_dataset, 
        [train_size, val_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )
    
    print(f"Dataset splits: Train={len(train_dataset)}, Val={len(val_dataset)}, Test={len(test_dataset)}")
    
    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=4)
    
    # Model configuration
    encoder_name = "resnet34"
    
    # Initialize Attention U-Net model
    # Initialize DeepLabV3+ model
    # PAN (Pyramid Attention Network)
    model = smp.PAN(
        encoder_name="resnet34",
        encoder_weights="imagenet",
        in_channels=3,
        classes=1,
        decoder_channels=256
    )

    model = model.to(device)
    
    print(f"Model: Pyramid attention network with {encoder_name} encoder")
    
    # Initialize loss and optimizer
    criterion = smp.losses.DiceLoss('binary')
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Train the model
    train_model(model, train_loader, val_loader, criterion, optimizer, device)
    
    # Evaluate on test set
    print("Evaluating on test set...")
    model.load_state_dict(torch.load('best_model.pth'))
    test_loss, test_iou = evaluate(model, test_loader, criterion, device)
    print(f'Test Loss: {test_loss:.4f}, Test IoU: {test_iou:.4f}')

if __name__ == '__main__':
    main()

Using device: cuda
Dataset splits: Train=12142, Val=2602, Test=2603


Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 187MB/s] 


Model: Pyramid attention network with resnet34 encoder


Epoch 1/5: 100%|██████████| 1518/1518 [03:13<00:00,  7.85it/s, loss=0.112, IoU=0.875]


Validation IoU: 0.8122


...

Validation IoU: 0.9031


Epoch 5/5: 100%|██████████| 1518/1518 [03:22<00:00,  7.48it/s, loss=0.0826, IoU=0.9]  


Validation IoU: 0.9033
Evaluating on test set...


/tmp/ipykernel_30/4292876771.py:80: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


Test Loss: 0.0809, Test IoU: 0.9031


## FPN

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import os
from pathlib import Path
import numpy as np
from tqdm import tqdm
import segmentation_models_pytorch as smp

def main():
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Dataset paths
    image_dir = '/kaggle/input/huron-dataset/Sliced_Images'
    mask_dir = '/kaggle/input/huron-dataset/Sliced_masks'
    
    # Transform for input images
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                           std=[0.229, 0.224, 0.225])
    ])
    
    # Create full dataset
    full_dataset = HuronDataset(image_dir, mask_dir, transform=transform)
    
    # Calculate splits
    total_size = len(full_dataset)
    train_size = int(0.7 * total_size)
    val_size = int(0.15 * total_size)
    test_size = total_size - train_size - val_size
    
    # Split dataset
    train_dataset, val_dataset, test_dataset = random_split(
        full_dataset, 
        [train_size, val_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )
    
    print(f"Dataset splits: Train={len(train_dataset)}, Val={len(val_dataset)}, Test={len(test_dataset)}")
    
    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=4)
    
    # Model configuration
    encoder_name = "resnet34"
    
    # Initialize Attention U-Net model
    # Initialize DeepLabV3+ model
    # PAN (Pyramid Attention Network)
    # FPN (Feature Pyramid Network)
    model = smp.FPN(
        encoder_name="resnet34",
        encoder_weights="imagenet",
        in_channels=3,
        classes=1,
    )

    model = model.to(device)
    
    print(f"Model: Feature Pyramid Network with {encoder_name} encoder")
    
    # Initialize loss and optimizer
    criterion = smp.losses.DiceLoss('binary')
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Train the model
    train_model(model, train_loader, val_loader, criterion, optimizer, device)
    
    # Evaluate on test set
    print("Evaluating on test set...")
    model.load_state_dict(torch.load('best_model.pth'))
    test_loss, test_iou = evaluate(model, test_loader, criterion, device)
    print(f'Test Loss: {test_loss:.4f}, Test IoU: {test_iou:.4f}')

if __name__ == '__main__':
    main()

Using device: cuda
Dataset splits: Train=12142, Val=2602, Test=2603
Model: Feature Pyramid Network with resnet34 encoder


Epoch 1/5: 100%|██████████| 1518/1518 [02:37<00:00,  9.66it/s, loss=0.107, IoU=0.878]


Validation IoU: 0.8939


Epoch 2/5: 100%|██████████| 1518/1518 [02:36<00:00,  9.70it/s, loss=0.1, IoU=0.886]   


Validation IoU: 0.8884


...

Validation IoU: 0.8925


Epoch 5/5: 100%|██████████| 1518/1518 [02:36<00:00,  9.72it/s, loss=0.0832, IoU=0.9]  


Validation IoU: 0.8804
Evaluating on test set...


/tmp/ipykernel_30/4272857622.py:80: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


Test Loss: 0.0939, Test IoU: 0.8935


## All model definitions

In [ ]:
import segmentation_models_pytorch as smp

# UNet
unet_model = smp.Unet(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1,
    decoder_channels=(256, 128, 64, 32, 16),  # default decoder channels
)

# UNet++
unetpp_model = smp.UnetPlusPlus(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1,
    decoder_channels=(256, 128, 64, 32, 16),
    decoder_attention_type="scse"  # Optional attention
)

# MANet
manet_model = smp.MAnet(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1,
    decoder_channels=(256, 128, 64, 32, 16),
    decoder_attention_type="scse"
)

# LinkNet
linknet_model = smp.Linknet(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1,
    decoder_channels=(256, 128, 64, 32, 16)
)

# FPN (Feature Pyramid Network)
fpn_model = smp.FPN(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1,
    decoder_channels=(256, 128, 64, 32, 16)  # can be adjusted
)

# PSPNet (Pyramid Scene Parsing Network)
psp_model = smp.PSPNet(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1,
    psp_out_channels=512,  # number of channels in PSP block
    psp_use_batchnorm=True
)

# PAN (Pyramid Attention Network)
pan_model = smp.PAN(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1,
    decoder_channels=(256, 128, 64, 32, 16)
)

# DeepLabV3
deeplabv3_model = smp.DeepLabV3(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1,
    decoder_channels=256  # number of channels in decoder
)

# DeepLabV3+
deeplabv3plus_model = smp.DeepLabV3Plus(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1,
    decoder_channels=256,
    decoder_atrous_rates=(12, 24, 36)  # atrous rates for ASPP module
)

# Common encoder options (can be used with any model above):
common_encoders = [
    "resnet18", "resnet34", "resnet50", "resnet101", "resnet152",
    "resnext50_32x4d", "resnext101_32x8d",
    "timm-efficientnet-b0", "timm-efficientnet-b1", "timm-efficientnet-b2",
    "timm-efficientnet-b3", "timm-efficientnet-b4", "timm-efficientnet-b5",
    "timm-efficientnet-b6", "timm-efficientnet-b7",
    "densenet121", "densenet169", "densenet201",
    "dpn68", "dpn98", "dpn131",
    "vgg11", "vgg13", "vgg16", "vgg19",
    "mobilenet_v2"
]

# Optional decoder attention types (for models that support it):
attention_types = [
    "scse",       # Concurrent Spatial and Channel Squeeze & Excitation
    "cbam",       # Convolutional Block Attention Module
    "eca",        # Efficient Channel Attention
    None          # No attention
]

# Example of how to check available encoders for a specific architecture
available_encoders = smp.Unet.get_encoder_names()
# print(available_encoders)  # Uncomment to see all available encoders

# Example of getting model with specific encoder
model_with_specific_encoder = smp.Unet(
    encoder_name="efficientnet-b0",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1
)